In [48]:
println(Threads.nthreads())

22


In [49]:
using IntervalArithmetic
using Statistics

In [50]:
include("./crispPCM/index.jl")
include("./evaluation/index.jl")
include("./method1/index.jl")
include("./method2/index.jl")
include("./method3/index.jl")
include("./method4/index.jl")
include("./intervalPCM/index.jl")
include("./twofoldInterval/index.jl")
include("./twofoldIntervalPCM/index.jl")
include("./utils.jl")

R (generic function with 1 method)

In [51]:
Random.seed!(124)

TaskLocalRNG()

In [52]:
SimulationCase = @NamedTuple{
    CR::T,
    A::Matrix{T},
    A₁::Matrix{Interval{T}},
    A₂::Matrix{Interval{T}}
} where {T <: Real}

function generateSimulationCases(
    n::Integer, 
    values::Vector{T},
    width::T,
    numOfCrispPCM::Integer, 
    intervalPCMsPerCrispPCM::Integer
)::Array{SimulationCase, 1} where {T <: Real}

    simulation_cases = SimulationCase[]

    for _ in 1:numOfCrispPCM
        # Generate a Crisp PCM
        crisp_pcm = generateConsistentCrispPCM(n, values)
        CR = consistencyRatio(crisp_pcm)

        for _ in 1:intervalPCMsPerCrispPCM
            # Generate an Interval PCM for each Crisp PCM
            interval_pcm_1 = randamizedIntervalPCM(crisp_pcm, width)
            interval_pcm_2 = randamizedIntervalPCM(crisp_pcm, width)
            # Append the Crisp PCM and its corresponding Interval PCM as a tuple to the list
            push!(simulation_cases, (CR=CR, A=crisp_pcm, A₁=interval_pcm_1, A₂=interval_pcm_2))
        end
    end

    return simulation_cases
end


generateSimulationCases (generic function with 2 methods)

In [53]:
SimulationResult = @NamedTuple{
    # A₁, A₂ の元となった crisp PCM
    A::Matrix{T},
    # 各 DM の与える区間 PCM
    A₁::Matrix{Interval{T}}, A₂::Matrix{Interval{T}},
    # method 1 ~ 4 の二重区間 PCM
    𝓐¹::Matrix{TwofoldInterval{T}}, 𝓐²::Matrix{TwofoldInterval{T}},
    𝓐³::Matrix{TwofoldInterval{T}}, 𝓐⁴::Matrix{TwofoldInterval{T}},
    # 𝓐ᵏ の二重区間の内側の区間
    𝓐¹⁻::Matrix{Interval{T}}, 𝓐²⁻::Matrix{Interval{T}},
    𝓐³⁻::Matrix{Interval{T}}, 𝓐⁴⁻::Matrix{Interval{T}},
    # 𝓐ᵏ の二重区間の外側の区間
    𝓐¹⁺::Matrix{Interval{T}}, 𝓐²⁺::Matrix{Interval{T}},
    𝓐³⁺::Matrix{Interval{T}}, 𝓐⁴⁺::Matrix{Interval{T}},
    # エラーメッセージ
    error::String
} where {T <: Real}

methodList = [method1, method2, method3, method4]

function runSimulation(
    case::SimulationCase{T}
    )::SimulationResult{T} where {T <: Real}
    𝓐 = Dict(); 𝓐⁻ = Dict(); 𝓐⁺ = Dict()
    
    try
        for k in 1:4
            𝓐[k] = methodList[k](case.A₁, case.A₂)
            𝓐⁻[k] = map(𝓐ᵢⱼ -> 𝓐ᵢⱼ[1], 𝓐[k])
            𝓐⁺[k] = map(𝓐ᵢⱼ -> 𝓐ᵢⱼ[2], 𝓐[k])
        end

        return (
            A=case.A, A₁=case.A₁, A₂=case.A₂,
            𝓐¹=𝓐[1], 𝓐²=𝓐[2], 𝓐³=𝓐[3], 𝓐⁴=𝓐[4],
            𝓐¹⁻=𝓐⁻[1], 𝓐²⁻=𝓐⁻[2], 𝓐³⁻=𝓐⁻[3], 𝓐⁴⁻=𝓐⁻[4],
            𝓐¹⁺=𝓐⁺[1], 𝓐²⁺=𝓐⁺[2], 𝓐³⁺=𝓐⁺[3], 𝓐⁴⁺=𝓐⁺[4],
            error=""
        )
    catch e
        n = size(case.A₁, 2)
        IPCM = fill(1..1, n, n)
        twofoldIPCM = fill((1..1, 1..1), n, n)
        return (
            A=case.A, A₁=case.A₁, A₂=case.A₂,
            𝓐¹=twofoldIPCM, 𝓐²=twofoldIPCM, 𝓐³=twofoldIPCM, 𝓐⁴=twofoldIPCM,
            𝓐¹⁻=IPCM, 𝓐²⁻=IPCM, 𝓐³⁻=IPCM, 𝓐⁴⁻=IPCM,
            𝓐¹⁺=IPCM, 𝓐²⁺=IPCM, 𝓐³⁺=IPCM, 𝓐⁴⁺=IPCM,
            error=string(e)
        )
    end
end

runSimulation (generic function with 1 method)

In [54]:
ConincidenceList = @NamedTuple{
    A₁_𝓐¹⁻::T, A₁_𝓐¹⁺::T, A₂_𝓐¹⁻::T, A₂_𝓐¹⁺::T,
    A₁_𝓐²⁻::T, A₁_𝓐²⁺::T, A₂_𝓐²⁻::T, A₂_𝓐²⁺::T,
    A₁_𝓐³⁻::T, A₁_𝓐³⁺::T, A₂_𝓐³⁻::T, A₂_𝓐³⁺::T,
    A₁_𝓐⁴⁻::T, A₁_𝓐⁴⁺::T, A₂_𝓐⁴⁻::T, A₂_𝓐⁴⁺::T,
    error::String
} where {T <: Real}

function calculateCoincidenceList(result::SimulationResult{T})::ConincidenceList{T} where {T <: Real}
    A₁ = result.A₁; A₂ = result.A₂
    𝓐¹⁻ = result.𝓐¹⁻; 𝓐¹⁺ = result.𝓐¹⁺
    𝓐²⁻ = result.𝓐²⁻; 𝓐²⁺ = result.𝓐²⁺
    𝓐³⁻ = result.𝓐³⁻; 𝓐³⁺ = result.𝓐³⁺
    𝓐⁴⁻ = result.𝓐⁴⁻; 𝓐⁴⁺ = result.𝓐⁴⁺
    try
        return (
            A₁_𝓐¹⁻=coincidenceIndex(A₁, 𝓐¹⁻), A₁_𝓐¹⁺=coincidenceIndex(A₁, 𝓐¹⁺),
            A₂_𝓐¹⁻=coincidenceIndex(A₂, 𝓐¹⁻), A₂_𝓐¹⁺=coincidenceIndex(A₂, 𝓐¹⁺),
            A₁_𝓐²⁻=coincidenceIndex(A₁, 𝓐²⁻), A₁_𝓐²⁺=coincidenceIndex(A₁, 𝓐²⁺),
            A₂_𝓐²⁻=coincidenceIndex(A₂, 𝓐²⁻), A₂_𝓐²⁺=coincidenceIndex(A₂, 𝓐²⁺),
            A₁_𝓐³⁻=coincidenceIndex(A₁, 𝓐³⁻), A₁_𝓐³⁺=coincidenceIndex(A₁, 𝓐³⁺),
            A₂_𝓐³⁻=coincidenceIndex(A₂, 𝓐³⁻), A₂_𝓐³⁺=coincidenceIndex(A₂, 𝓐³⁺),
            A₁_𝓐⁴⁻=coincidenceIndex(A₁, 𝓐⁴⁻), A₁_𝓐⁴⁺=coincidenceIndex(A₁, 𝓐⁴⁺),
            A₂_𝓐⁴⁻=coincidenceIndex(A₂, 𝓐⁴⁻), A₂_𝓐⁴⁺=coincidenceIndex(A₂, 𝓐⁴⁺),
            error=""
        )
    catch e
        return (
            A₁_𝓐¹⁻=NaN, A₁_𝓐¹⁺=NaN, A₂_𝓐¹⁻=NaN, A₂_𝓐¹⁺=NaN,
            A₁_𝓐²⁻=NaN, A₁_𝓐²⁺=NaN, A₂_𝓐²⁻=NaN, A₂_𝓐²⁺=NaN,
            A₁_𝓐³⁻=NaN, A₁_𝓐³⁺=NaN, A₂_𝓐³⁻=NaN, A₂_𝓐³⁺=NaN,
            A₁_𝓐⁴⁻=NaN, A₁_𝓐⁴⁺=NaN, A₂_𝓐⁴⁻=NaN, A₂_𝓐⁴⁺=NaN,
            error=string(e)
        )
    end
end

calculateCoincidenceList (generic function with 1 method)

In [55]:
function averageConincidenceLists(conincidenceLists::Vector{ConincidenceList{T}}) where {T <: Real}
    if length(conincidenceLists) == 0
        throw(ArgumentError("Empty list of ConincidenceLists"))
    end

    total_𝓐¹⁻ = zero(T); total_𝓐¹⁺ = zero(T)
    total_𝓐²⁻ = zero(T); total_𝓐²⁺ = zero(T)
    total_𝓐³⁻ = zero(T); total_𝓐³⁺ = zero(T)
    total_𝓐⁴⁻ = zero(T); total_𝓐⁴⁺ = zero(T)

    count = 0

    for list in conincidenceLists
        if list.error != "" continue end

        count += 1
        total_𝓐¹⁻ += list.A₁_𝓐¹⁻ + list.A₂_𝓐¹⁻
        total_𝓐¹⁺ += list.A₁_𝓐¹⁺ + list.A₂_𝓐¹⁺
        total_𝓐²⁻ += list.A₁_𝓐²⁻ + list.A₂_𝓐²⁻
        total_𝓐²⁺ += list.A₁_𝓐²⁺ + list.A₂_𝓐²⁺
        total_𝓐³⁻ += list.A₁_𝓐³⁻ + list.A₂_𝓐³⁻
        total_𝓐³⁺ += list.A₁_𝓐³⁺ + list.A₂_𝓐³⁺
        total_𝓐⁴⁻ += list.A₁_𝓐⁴⁻ + list.A₂_𝓐⁴⁻
        total_𝓐⁴⁺ += list.A₁_𝓐⁴⁺ + list.A₂_𝓐⁴⁺
    end
    count *= 2

    return (
        𝓐¹⁻ = total_𝓐¹⁻ / count, 𝓐¹⁺ = total_𝓐¹⁺ / count,
        𝓐²⁻ = total_𝓐²⁻ / count, 𝓐²⁺ = total_𝓐²⁺ / count,
        𝓐³⁻ = total_𝓐³⁻ / count, 𝓐³⁺ = total_𝓐³⁺ / count,
        𝓐⁴⁻ = total_𝓐⁴⁻ / count, 𝓐⁴⁺ = total_𝓐⁴⁺ / count,
    )
end

averageConincidenceLists (generic function with 1 method)

In [56]:
function runSimulate(
    n::Integer, 
    values::Vector{T},
    width::T,
    numOfCrispPCM::Integer, 
    intervalPCMsPerCrispPCM::Integer
    ) where {T <: Real}
    cases = generateSimulationCases(5, [1.0, 3, 5, 7], 0.5, 10, 100)
    results = Vector{SimulationResult}(undef, length(cases))
    Threads.@threads for i in 1:length(cases)
        results[i] = runSimulation(cases[i])
    end
    conincidenceList = calculateCoincidenceList.(results)
    return averageConincidenceLists(conincidenceList)
end

runSimulate (generic function with 1 method)

In [63]:
print("n = 5")
println(runSimulate(5, [1.0, 3, 5, 7, 9], 0.1, 10, 100))
println(runSimulate(5, [1.0, 3, 5, 7, 9], 0.5, 10, 100))
println(runSimulate(5, [1.0, 3, 5, 7, 9], 1.0, 10, 100))
print("n = 6")
println(runSimulate(6, [1.0, 3, 5, 7, 9], 0.1, 10, 100))
println(runSimulate(6, [1.0, 3, 5, 7, 9], 0.5, 10, 100))
println(runSimulate(6, [1.0, 3, 5, 7, 9], 1.0, 10, 100))
print("n = 7")
println(runSimulate(7, [1.0, 3, 5, 7, 9], 0.1, 10, 100))
println(runSimulate(7, [1.0, 3, 5, 7, 9], 0.5, 10, 100))
println(runSimulate(7, [1.0, 3, 5, 7, 9], 1.0, 10, 100))
print("n = 8")
println(runSimulate(8, [1.0, 3, 5, 7, 9], 0.1, 10, 100))
println(runSimulate(8, [1.0, 3, 5, 7, 9], 0.5, 10, 100))
println(runSimulate(8, [1.0, 3, 5, 7, 9], 1.0, 10, 100))

(𝓐¹⁻ = 0.00400593778500724, 𝓐¹⁺ = 0.2883845891268793, 𝓐²⁻ = 1.4498756059818845e-16, 𝓐²⁺ = 0.3587593929145373, 𝓐³⁻ = 0.0, 𝓐³⁺ = 0.3076141825851131, 𝓐⁴⁻ = 0.0, 𝓐⁴⁺ = 0.38804418886858866)


(𝓐¹⁻ = 0.0033389904150032037, 𝓐¹⁺ = 0.2890845874680869, 𝓐²⁻ = 1.4234681514203832e-16, 𝓐²⁺ = 0.3464735391240684, 𝓐³⁻ = 0.0, 𝓐³⁺ = 0.29445918327936005, 𝓐⁴⁻ = 0.0, 𝓐⁴⁺ = 0.37766237255198765)


MethodError: MethodError: no method matching runSimulate(::Int64, ::Vector{Float64}, ::Int64, ::Int64, ::Int64)

Closest candidates are:
  runSimulate(::Integer, ::Vector{T}, !Matched::T, ::Integer, ::Integer) where T<:Real
   @ Main /workspaces/illknownpcm/src/20230622.ipynb:1
